In [ ]:
from __future__ import annotations

import itertools

import numpy as np
from monty.serialization import dumpfn, loadfn

In [ ]:
for functional in ("PBE", "r2SCAN"):
    data = loadfn(f"MatPES-{functional}-2025.1.json.gz")
    coh = loadfn(f"MatPES-{functional}-atoms.json.gz")
    mp_elements = [d for d in data if d["nelements"] == 1 and d["matpes_id"].startswith("mp")]
    el_atomic = {d["elements"][0]: d["energy"] for d in coh}
    str_atomic = {d["elements"][0]: d["structure"] for d in coh}

    get_element = lambda d: d["elements"][0]

    mp_elements = sorted(mp_elements, key=get_element)

    entries = {}

    for el, group in itertools.groupby(mp_elements, key=get_element):
        e = min(group, key=lambda d: d["energy"] / d["nsites"])  # Get ground state structure
        e["energy_per_atom"] = e["energy"] / e["nsites"]  # Add energy per atom key for simplicity.
        e["energy_atomic"] = el_atomic[el]
        e["structure_atomic"] = str_atomic[el]
        try:
            del e["formation_energy_per_atom"]
        except KeyError:
            pass
        assert np.allclose(e["energy_per_atom"] - e["energy_atomic"], e["cohesive_energy_per_atom"]), (
            f"{e['energy_per_atom'] - e['energy_atomic']} != {e['cohesive_energy_per_atom']}"
        )
        entries[el] = e
    assert len(entries) == 89
    dumpfn(entries, f"MatPES-{functional}-Element-Refs.json.gz")